In [1]:
import os
os.chdir('..')

from puzzle_generator import *

In [2]:
width = 6
height = 6
puzzle = RectangularPuzzle(
    width=width,
    height=height
)

puzzle.solving_timeout = 30

puzzle.naming = {
    None: '_',
    'e': ' ',
    'q': 'Q',
    'k': 'K',
    'r': 'R',
    'b': 'B',
}

puzzle.domain_program += """
    #const num_attacks_per_piece=2.
    value(e;q;k;r;b).
    1 { guessed_number(num_knights,2..12) } 1.
    1 { guessed_number(num_queens,2..12) } 1.
    1 { guessed_number(num_rooks,2..12) } 1.
    1 { guessed_number(num_bishops,2..12) } 1.
    guessed_number('num_attacks_per_piece',num_attacks_per_piece).
"""

puzzle.puzzle_program += """
    { puzzle(C,V) : cell(C), value(V) }.
    #minimize { 1,puzzle(C,V) : puzzle(C,V) }.
    :- puzzle(C,e), cell(C).
    :- guessed_number(num_knights,N),
        N { puzzle(C,k) } N.
    :- guessed_number(num_queens,N),
        N { puzzle(C,q) } N.
    :- guessed_number(num_rooks,N),
        N { puzzle(C,r) } N.
    :- guessed_number(num_bishops,N),
        N { puzzle(C,b) } N.
"""

puzzle.solution_program += enc_library['knights_move']
puzzle.solution_program += enc_library['queens_move']
puzzle.solution_program += enc_library['rooks_move']
puzzle.solution_program += enc_library['bishops_move']
puzzle.solution_program += """
    :- guessed_number(num_knights,N),
        not N { solution(C,k) } N.
    :- guessed_number(num_queens,N),
        not N { solution(C,q) } N.
    :- guessed_number(num_rooks,N),
        not N { solution(C,r) } N.
    :- guessed_number(num_bishops,N),
        not N { solution(C,b) } N.
    piece_attacks(C1,C2) :-
        solution(C1,k), knights_move(C1,C2).
    piece_attacks(C1,C2) :-
        solution(C1,q), queens_move(C1,C2).
    piece_attacks(C1,C2) :-
        solution(C1,r), rooks_move(C1,C2).
    piece_attacks(C1,C2) :-
        solution(C1,b), bishops_move(C1,C2).
    :- solution(C1,P), P != e,
        N = num_attacks_per_piece,
        not N { piece_attacks(C2,C1) } N.
"""

In [3]:
puzzle.generate(
    verbose=True,
    precompute_solution=False,
)

....
Stopped after solving timeout..
Solving time: 30.79 seconds



In [4]:
print(puzzle.pretty_repr_puzzle())

|_|_|_|_|_|_|
|_|_|_|_|_|_|
|_|_|Q|Q|_|_|
|_|_|Q|_|_|_|
|_|_|_|_|_|_|
|R|Q|_|_|_|B|

'num_attacks_per_piece' = 2
num_bishops = 3
num_rooks = 3
num_queens = 5
num_knights = 2


In [5]:
print(puzzle.pretty_repr_solution())

|K| | | | |K|
|R| | | | |Q|
|B| |Q|Q| | |
|B| |Q| | | |
| | | | |R| |
|R|Q| | | |B|
